<a href="https://colab.research.google.com/github/abelalanoca/P.MAGNO/blob/main/TP1/Limpieza_y_estructuraacion_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [192]:
from google.colab import files
from google.colab import drive
import pandas as pd

# 1️⃣ Subir archivo desde tu PC
uploaded = files.upload()

# 2️⃣ Montar Google Drive (si quieres guardar/leer desde ahí)
drive.mount("/content/drive")

# 3️⃣ Leer hoja 1 del archivo subido
nombre_archivo = list(uploaded.keys())[0]
df_sheet1 = pd.read_excel(nombre_archivo, sheet_name=0)

# 4️⃣ Ver primeras filas
df_sheet1.head()



Saving B-2201-jn2025.XLS to B-2201-jn2025.XLS
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87
0,Balance General por Empresa Bancaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-06-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,(En miles de soles),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Activo,Banco BBVA Perú,NaN,NaN,NaN,BANCOM,NaN,NaN,NaN,Banco de Crédito del Perú,...,NaN,NaN,NaN,Banco de Crédito con Sucursales en el \nExterior,NaN,NaN,NaN,Total Banca Múltiple Incluye Sucursales en el ...,NaN,NaN


In [193]:
def reemplazar_bancos_especificos(df):
    """
    Normaliza nombres específicos de bancos en el DataFrame.
    Devuelve un nuevo DataFrame (no modifica el original).
    """
    df_1 = df.copy()

    reemplazos = {
        "COMPARTAMOS BANCO": "Compartamos Banco*",
        "BANCO DE COMERCIO": "BANCOM"
    }

    # Reemplazar solo si es texto
    df_1 = df_1.applymap(
        lambda x: reemplazos.get(str(x).strip().upper(), x)
        if isinstance(x, str) else x
    )

    return df_1

# 📌 Usar la función
df_1 = reemplazar_bancos_especificos(df_sheet1)


/tmp/ipython-input-1045875759.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_1 = df_1.applymap(


In [194]:
def extraer_listas_balance(df):
    # Convertir primera columna a formato estándar (texto limpio y en mayúsculas)
    primera_col = df.iloc[:, 0].astype(str).str.strip().str.upper()

    # --- Cuentas del Activo ---
    idx_activo = primera_col[primera_col == "ACTIVO"].index[0]
    idx_total_activo = primera_col[primera_col == "TOTAL ACTIVO"].index[0]

    cuentas_del_activo = (
        df.iloc[idx_activo:idx_total_activo + 1, 0]
        .dropna().astype(str).str.strip().tolist()
    )
    if cuentas_del_activo and cuentas_del_activo[0].upper() == "ACTIVO":
        cuentas_del_activo = cuentas_del_activo[1:]

    # --- Lista de Pasivos ---
    idx_pasivo = primera_col[primera_col == "PASIVO"].index[0]
    idx_otras_conting = primera_col[primera_col == "OTRAS CUENTAS CONTINGENTES"].index[0]

    lista_pasivos = (
        df.iloc[idx_pasivo:idx_otras_conting + 1, 0]
        .dropna().astype(str).str.strip().tolist()
    )
    lista_pasivos = [item for item in lista_pasivos if item.upper() != "PASIVO"]

    # --- Bancos en la fila "Activo" ---
    bancos_activo = (
        df.iloc[idx_activo, 1:]
        .dropna().astype(str).str.strip().tolist()
    )
    bancos_activo = [b for b in bancos_activo if b.upper() != "ACTIVO"]

    return cuentas_del_activo, lista_pasivos, bancos_activo


# 4️⃣ Ejecutar función y mostrar resultados
cuentas_del_activo, lista_pasivos, bancos_activo = extraer_listas_balance(df_sheet1)

print("📌 Cuentas del Activo:")
print(cuentas_del_activo)

print("\n📌 Lista de Pasivos:")
print(lista_pasivos)

print("\n📌 Bancos en Activo:")
print(bancos_activo)


📌 Cuentas del Activo:
['DISPONIBLE', 'Caja', 'Bancos y Corresponsales', 'Canje', 'Otros', 'FONDOS INTERBANCARIOS', 'INVERSIONES NETAS DE PROVISIONES', 'Inversiones a Valor Razonable con Cambios en Resultados', 'Inversiones Disponibles para la Venta', 'Inversiones a Vencimiento', 'Inversiones en Subsidiarias, Asociadas y Negocios Conjuntos', 'Inversiones en Commodities', 'Provisiones', 'CRÉDITOS NETOS DE PROVISIONES Y DE INGRESOS NO DEVENGADOS', 'Vigentes', 'Cuentas Corrientes', 'Tarjetas de Crédito', 'Descuentos', 'Factoring', 'Préstamos', 'Arrendamiento Financiero', 'Hipotecarios para Vivienda', 'Comercio Exterior', 'Créditos por Liquidar', 'Otros', 'Refinanciados y Reestructurados', 'Atrasados', 'Vencidos', 'En Cobranza Judicial', 'Provisiones', 'Intereses y Comisiones no Devengados', 'CUENTAS POR COBRAR NETAS DE PROVISIONES', 'RENDIMIENTOS POR COBRAR', 'Disponible', 'Fondos Interbancarios', 'Inversiones', 'Créditos', 'Cuentas por Cobrar', 'BIENES REALIZABLES, RECIBIDOS EN PAGO Y ADJ

In [195]:
# 2️⃣ Función para extraer fecha con formato Mes-Día (ej: Feb-31)
def extraer_fecha(df):
    # Buscar índice de la fila que contiene "Balance General por Empresa Bancaria"
    idx_balance = df[df.astype(str).apply(lambda row: row.str.contains("Balance General por Empresa Bancaria", case=False, na=False)).any(axis=1)].index[0]

    # Obtener la celda de abajo (donde está la fecha)
    fecha_cruda = df.iloc[idx_balance + 1].dropna().astype(str).tolist()[0]

    # Convertir a datetime y formatear como Mes-Día
    try:
        fecha_dt = pd.to_datetime(fecha_cruda, errors='coerce')
        if pd.notnull(fecha_dt):
            fecha_formateada = fecha_dt.strftime("%b-%d")  # Ejemplo: Feb-31
        else:
            fecha_formateada = fecha_cruda
    except:
        fecha_formateada = fecha_cruda

    return [fecha_formateada]  # Lista con un solo elemento

# 3️⃣ Ejecutar función
Fecha = extraer_fecha(df_sheet1)

# 4️⃣ Mostrar lista
print("📅 Lista Fecha:", Fecha)

📅 Lista Fecha: ['Jun-30']


In [196]:
import pandas as pd

def crear_df_2(Fecha, bancos_activo, cuentas_del_activo, lista_pasivos):
    # Función interna para filtrar nulos y vacíos
    def limpiar_lista(lista):
        return [item for item in lista if pd.notnull(item) and str(item).strip() != ""]

    # Limpiar listas
    cuentas_del_activo = limpiar_lista(cuentas_del_activo)
    lista_pasivos = limpiar_lista(lista_pasivos)
    bancos_activo = limpiar_lista(bancos_activo)

    # Combinar todas las cuentas
    columnas_cuentas = cuentas_del_activo + lista_pasivos

    # Crear estructura vacía
    df_2 = pd.DataFrame(columns=["MES", "ENTIDAD", "MONEDA"] + columnas_cuentas)

    # Listas para columnas fijas
    entidades_repetidas = []
    monedas_repetidas = []
    meses_repetidos = []

    for banco in bancos_activo:
        for moneda in ["MN", "ME", "TOTAL"]:
            entidades_repetidas.append(banco)
            monedas_repetidas.append(moneda)
            meses_repetidos.append(Fecha[0])  # Siempre misma fecha para todos

    # Llenar las columnas fijas
    df_2["MES"] = meses_repetidos
    df_2["ENTIDAD"] = entidades_repetidas
    df_2["MONEDA"] = monedas_repetidas

    return df_2

# Crear df_2 limpio
df_2 = crear_df_2(Fecha, bancos_activo, cuentas_del_activo, lista_pasivos)
df_2

,MES,ENTIDAD,MONEDA,DISPONIBLE,Caja,Bancos y Corresponsales,Canje,Otros,FONDOS INTERBANCARIOS,INVERSIONES NETAS DE PROVISIONES,...,Reservas,Ajustes al Patrimonio,Resultados Acumulados,Resultado Neto del Ejercicio,TOTAL PASIVO Y PATRIMONIO,CONTINGENTES,"Avales, Cartas Fianza, Cartas de Crédito y Aceptaciones Bancarias",Líneas de Crédito no Utilizadas y Créditos Concedidos no Desembolsados,Instrumentos Financieros Derivados,Otras Cuentas Contingentes
0,Jun-30,Banco BBVA Perú,MN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jun-30,Banco BBVA Perú,ME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jun-30,Banco BBVA Perú,TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Jun-30,BANCOM,MN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Jun-30,BANCOM,ME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Jun-30,Banco de Crédito con Sucursales en el \nExterior,ME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,Jun-30,Banco de Crédito con Sucursales en el \nExterior,TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,Jun-30,Total Banca Múltiple Incluye Sucursales en el ...,MN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,Jun-30,Total Banca Múltiple Incluye Sucursales en el ...,ME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [197]:
def llenar_df_2_pasivo_matriz(df_1, df_2, bancos_activo, lista_pasivos):
    # Función para limpiar listas
    def limpiar_lista(lista):
        return [item for item in lista if pd.notnull(item) and str(item).strip() != ""]

    # Limpiar las listas de entrada
    bancos_activo = limpiar_lista(bancos_activo)
    lista_pasivos = limpiar_lista(lista_pasivos)

    # 1️⃣ Limpiar textos en el DataFrame
    df_limpio = df_1.copy()
    df_limpio = df_limpio.applymap(lambda x: str(x).strip() if pd.notnull(x) else x)

    # 2️⃣ Localizar índice de "PASIVO"
    idx_pasivo = df_limpio[df_limpio.iloc[:, 0].str.upper() == "PASIVO"].index[0]

    # 3️⃣ Filas con cuentas del pasivo
    filas_cuentas = {}
    for cuenta in lista_pasivos:
        match = df_limpio.index[df_limpio.iloc[:, 0].str.upper() == cuenta.upper()].tolist()
        if match:
            filas_cuentas[cuenta] = match[0]

    # 4️⃣ Columnas con bancos
    columnas_bancos = {}
    for banco in bancos_activo:
        match = df_limpio.columns[df_limpio.iloc[idx_pasivo, :].astype(str) == banco].tolist()
        if match:
            col_index = df_limpio.columns.get_loc(match[0])
            columnas_bancos[banco] = col_index

    # 5️⃣ Recorrer banco-cuenta y llenar valores MN, ME, TOTAL
    for banco, col_idx in columnas_bancos.items():
        for cuenta, fila_idx in filas_cuentas.items():
            valores = df_1.iloc[fila_idx, col_idx:col_idx+3].tolist()
            for moneda, valor in zip(["MN", "ME", "TOTAL"], valores):
                df_2.loc[(df_2["ENTIDAD"] == banco) & (df_2["MONEDA"] == moneda), cuenta] = valor

    return df_2


In [198]:
# 1️⃣ Extraer listas ya filtradas
cuentas_del_activo, lista_pasivos, bancos_activo = extraer_listas_balance(df_1)

# 2️⃣ Crear df_2 vacío
df_2 = crear_df_2(Fecha, bancos_activo, cuentas_del_activo, lista_pasivos)

# 3️⃣ Llenar primero ACTIVO
df_2 = llenar_df_2_activo_matriz(df_1, df_2, bancos_activo, cuentas_del_activo)

# 4️⃣ Luego PASIVO
df_2 = llenar_df_2_pasivo_matriz(df_1, df_2, bancos_activo, lista_pasivos)

# 5️⃣ Mostrar resultados
df_2




/tmp/ipython-input-4121894297.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df_limpio.applymap(lambda x: str(x).strip() if pd.notnull(x) else x)
/tmp/ipython-input-2879807518.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df_limpio.applymap(lambda x: str(x).strip() if pd.notnull(x) else x)


,MES,ENTIDAD,MONEDA,DISPONIBLE,Caja,Bancos y Corresponsales,Canje,Otros,FONDOS INTERBANCARIOS,INVERSIONES NETAS DE PROVISIONES,...,Reservas,Ajustes al Patrimonio,Resultados Acumulados,Resultado Neto del Ejercicio,TOTAL PASIVO Y PATRIMONIO,CONTINGENTES,"Avales, Cartas Fianza, Cartas de Crédito y Aceptaciones Bancarias",Líneas de Crédito no Utilizadas y Créditos Concedidos no Desembolsados,Instrumentos Financieros Derivados,Otras Cuentas Contingentes
0,Jun-30,Banco BBVA Perú,MN,2809934.811,2043287.409,456078.644,50195.75,260373.007,62000,8154389.076,...,2622071.734,203855.687,0,1107166.587,72848013.626,26329635.493,9540952.52,16696449.941,92233.03,0
1,Jun-30,Banco BBVA Perú,ME,7886397.251,803611.794,3904568.819,25185.208,3153031.428,17710,7500246.349,...,0,-11332.512,0,0,35996598.023,75110649.598,11898718.928,4728405.123,58477858.345,5667.2
2,Jun-30,Banco BBVA Perú,TOTAL,10696332.062,2846899.203,4360647.463,75380.959,3413404.436,79710,15654635.425,...,2622071.734,192523.175,0,1107166.587,108844611.65,101440285.091,21439671.449,21424855.065,58570091.376,5667.2
3,Jun-30,BANCOM,MN,85103.979,52814.051,30741.095,1353.404,195.427,60000,60447.952,...,35696.439,0.696,0,6245.043,1786403.86,88791.828,88791.828,0,0,0
4,Jun-30,BANCOM,ME,126029.485,22376.416,103564.519,88.55,0,0,0,...,0,0,0,0,282945.536,159571.02,11062.956,148508.064,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Jun-30,Banco de Crédito con Sucursales en el \nExterior,ME,33194982.326,1479243.945,31381639.249,45389.033,288710.097,0,4090625.295,...,0,13596.474,722293.767,0,75675435.694,110386434.54,16211927.61,47704759.22,46468345.824,1401.885
62,Jun-30,Banco de Crédito con Sucursales en el \nExterior,TOTAL,37158738.772,3878478.067,32867530.959,122868.607,289861.136,215000,29775119.097,...,6125451.89,-484162.08,1842367.071,3286882.513,190742892.162,136224331.732,21467769.107,68251113.298,46468345.824,37103.501
63,Jun-30,Total Banca Múltiple Incluye Sucursales en el ...,MN,16410141.732,7416149.432,8166613.714,178101.913,649276.65,817700,64059847.308,...,14277157.34,-240555.24,2338973.346,7072338.237,361854491.175,100241924.197,25085972.892,68070500.341,6943734.839,141716.119
64,Jun-30,Total Banca Múltiple Incluye Sucursales en el ...,ME,74800740.486,4257650.68,63745429.756,108171.938,6689488.095,251482,15861694.02,...,0,120264.006,722293.767,0,186293027.51,315280709.711,41771894.097,88013864.694,185485095.124,9855.783


In [199]:
# Limpiar encabezados para que solo contengan letras (y espacios)
df_2.columns = df_2.columns.map(lambda x: ''.join(ch for ch in str(x) if ch.isalpha() or ch.isspace()))

In [200]:
df_2

,MES,ENTIDAD,MONEDA,DISPONIBLE,Caja,Bancos y Corresponsales,Canje,Otros,FONDOS INTERBANCARIOS,INVERSIONES NETAS DE PROVISIONES,...,Reservas,Ajustes al Patrimonio,Resultados Acumulados,Resultado Neto del Ejercicio,TOTAL PASIVO Y PATRIMONIO,CONTINGENTES,Avales Cartas Fianza Cartas de Crédito y Aceptaciones Bancarias,Líneas de Crédito no Utilizadas y Créditos Concedidos no Desembolsados,Instrumentos Financieros Derivados,Otras Cuentas Contingentes
0,Jun-30,Banco BBVA Perú,MN,2809934.811,2043287.409,456078.644,50195.75,260373.007,62000,8154389.076,...,2622071.734,203855.687,0,1107166.587,72848013.626,26329635.493,9540952.52,16696449.941,92233.03,0
1,Jun-30,Banco BBVA Perú,ME,7886397.251,803611.794,3904568.819,25185.208,3153031.428,17710,7500246.349,...,0,-11332.512,0,0,35996598.023,75110649.598,11898718.928,4728405.123,58477858.345,5667.2
2,Jun-30,Banco BBVA Perú,TOTAL,10696332.062,2846899.203,4360647.463,75380.959,3413404.436,79710,15654635.425,...,2622071.734,192523.175,0,1107166.587,108844611.65,101440285.091,21439671.449,21424855.065,58570091.376,5667.2
3,Jun-30,BANCOM,MN,85103.979,52814.051,30741.095,1353.404,195.427,60000,60447.952,...,35696.439,0.696,0,6245.043,1786403.86,88791.828,88791.828,0,0,0
4,Jun-30,BANCOM,ME,126029.485,22376.416,103564.519,88.55,0,0,0,...,0,0,0,0,282945.536,159571.02,11062.956,148508.064,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Jun-30,Banco de Crédito con Sucursales en el \nExterior,ME,33194982.326,1479243.945,31381639.249,45389.033,288710.097,0,4090625.295,...,0,13596.474,722293.767,0,75675435.694,110386434.54,16211927.61,47704759.22,46468345.824,1401.885
62,Jun-30,Banco de Crédito con Sucursales en el \nExterior,TOTAL,37158738.772,3878478.067,32867530.959,122868.607,289861.136,215000,29775119.097,...,6125451.89,-484162.08,1842367.071,3286882.513,190742892.162,136224331.732,21467769.107,68251113.298,46468345.824,37103.501
63,Jun-30,Total Banca Múltiple Incluye Sucursales en el ...,MN,16410141.732,7416149.432,8166613.714,178101.913,649276.65,817700,64059847.308,...,14277157.34,-240555.24,2338973.346,7072338.237,361854491.175,100241924.197,25085972.892,68070500.341,6943734.839,141716.119
64,Jun-30,Total Banca Múltiple Incluye Sucursales en el ...,ME,74800740.486,4257650.68,63745429.756,108171.938,6689488.095,251482,15861694.02,...,0,120264.006,722293.767,0,186293027.51,315280709.711,41771894.097,88013864.694,185485095.124,9855.783


In [201]:
# Guardar el DataFrame como archivo Excel
df_2.to_excel("mi_archivo_limpio.xlsx", index=False)

# Descargar el archivo a tu PC
from google.colab import files
files.download("mi_archivo_limpio.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>